In [ ]:
#importing data analysis libraries and training dataset 
import pandas as pd 
import numpy as np 
df_training = pd.read_csv('udemy_entrenamiento.csv')
df_test = pd.read_csv('udemy_prueba.csv')
df_training.head(10) 
df_test.head(10) 

In [ ]:
#data split
#step 1: separating response variable -target- from predictor variables -inputs-
target_training = df_training['bestseller'] 
#step 2: data cleansing and prepping
inputs_training =  df_training[['is_practice_test_course', 'captions', 'rating', 'num_published_lectures', 'num_published_practice_tests', 'locale', 'instructional_level_simple', 'published_date', 'last_update_date', 'category', 'list_price', 'discount_price']]
inputs_training[inputs_training.columns[10:]] = inputs_training[inputs_training.columns[10:]].replace('[\$]', '', regex=True).astype(float)
inputs_training['discount_price'] = inputs_training['discount_price'].fillna(0)
inputs_training[inputs_training.columns[[7,8]]] = inputs_training[inputs_training.columns[[7,8]]].replace('[\-]', '', regex=True).astype(float)
inputs_training['last_update_date'] = inputs_training['last_update_date'].fillna(0)
print(inputs_training.info())
inputs_training.head()

In [ ]:
#label encoding inputs: find object datatype columns and create list
objList = inputs_training.select_dtypes(include = "object").columns
#print (objList)

#Label Encoding for object to numeric conversion, start with inputs
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for feat in objList:
    inputs_training[feat] = le.fit_transform(inputs_training[feat].astype(str))

print (inputs_training.info())

#then le target
le = LabelEncoder()
target_training_dummy= pd.DataFrame(le.fit_transform(target_training))
target_training_dummy.columns=['bestseller']

#inputs_training.head()
#label encoding alternative method:  (creates dummy variables for every category, not useful)
'''from sklearn.preprocessing import LabelEncoder
cat_cols = [
    f for f in inputs_training.columns if inputs_training[f].dtype == 'object'
]
inputs_training_dummies = pd.get_dummies(inputs_training, columns=cat_cols)'''

In [ ]:
#train model & split data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs_training,target_training_dummy,test_size=0.2)
#import and apply decision tree
from sklearn import tree
model = tree.DecisionTreeClassifier()
model.fit(X_train,y_train)
model.score(X_test,y_test)

In [ ]:
#insert test data
df_test_real = df_test.drop('bestseller', axis='columns')
df_test_real =  df_test_real[['is_practice_test_course', 'captions', 'rating', 'num_published_lectures', 'num_published_practice_tests', 'locale', 'instructional_level_simple', 'published_date', 'last_update_date', 'category', 'list_price', 'discount_price']]
df_test_real[df_test_real.columns[10:]] = df_test_real[df_test_real.columns[10:]].replace('[\$]', '', regex=True).astype(float)
df_test_real['discount_price'] = df_test_real['discount_price'].fillna(0)
df_test_real[df_test_real.columns[[7,8]]] = df_test_real[df_test_real.columns[[7,8]]].replace('[\-]', '', regex=True).astype(float)
df_test_real['last_update_date'] = df_test_real['last_update_date'].fillna(0)
print(df_test_real.info())
df_test_real.head()

In [ ]:
#label encoding for test data: find object datatype columns and create list
objList_test = df_test_real.select_dtypes(include = "object").columns
#print (objList)

#Label Encoding for object to numeric conversion, start with inputs
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for feat in objList:
    df_test_real[feat] = le.fit_transform(df_test_real[feat].astype(str))

print (df_test_real.info())

In [ ]:
#predict values with test data
model.predict(df_test_real) #here we get predicted values
#create model object
predictions = model.predict(df_test_real)
# finally, insert new course data (ids), append it to new dataframe with predicted values and export it to csv for submission 
df_solucion = pd.read_csv('udemy_ejemplo_solucion.csv')
df_solucion_2 = df_solucion['id']
predictions_df=pd.DataFrame(predictions)
predictions_new_df=predictions_df.join(df_solucion_2)
predictions_new_df=predictions_new_df.iloc[:, [1,0]]
predictions_new_df.columns=['id','bestseller']
predictions_new_df
predictions_new_df.to_csv('predictions_udemy_sklearn.csv', index=False,header=True)
